In [5]:
import servicebus_funcs
import var_funcs
from set_environment import config, current_config
import asyncio
import validation_nsip_project

In [8]:
_NAMESPACE = current_config['servicebus_namespace_odw']
_SUBSCRIPTION = config['global']['nsip-project-entity']
_TOPIC = config['global']['nsip-project-entity']
_CREDENTIAL = var_funcs.CREDENTIAL
_MAX_MESSAGE_COUNT = config['global']['max_message_count']
_STORAGE = current_config['storage_account']
_CONTAINER = current_config['storage_container']
_ENTITY = config['global']['nsip-project-entity']
_CURRENT_DATE = var_funcs.CURRENT_DATE
_UTC_TIMESTAMP = var_funcs.UTC_TIMESTAMP
_FILENAME = f"{_ENTITY}/{_CURRENT_DATE}/{_ENTITY}_{_UTC_TIMESTAMP}.json"

In [ ]:
servicebus_funcs.get_messages(_NAMESPACE,
                              _CREDENTIAL,
                              _TOPIC,
                              _SUBSCRIPTION,
                              _MAX_MESSAGE_COUNT)

In [2]:
async def validatedata():
    await validation_nsip_project.validate()
    # Simulating failure
    # Uncomment the below line to simulate an operation failure
    # raise Exception("Operation 1 failed")

In [3]:
async def sendtostorage():
    await servicebus_funcs.send_to_storage(_STORAGE, _CREDENTIAL, _CONTAINER, _FILENAME, data=validatedata())

In [4]:
async def process_message(msg, processed_messages):
    # Process the received message
    await validatedata()
    await sendtostorage()

    # Store the processed message
    processed_messages.append(msg)

In [ ]:
async def perform_additional_functions():
    # Simulating additional functions
    await   # Simulating additional processing
    # Simulating failure
    # Uncomment the below line to simulate a failure in additional functions
    # raise Exception("Additional functions failed")

In [ ]:
async def receive_messages():
    processed_messages = []

    # Your Azure Service Bus client setup
    servicebus_client = ServiceBusClient.from_connection_string("<connection_string>")
    async with servicebus_client:
        receiver = servicebus_client.get_subscription_receiver(
            topic_name=_TOPIC,
            subscription_name="<subscription_name>",
            max_wait_time=5,
        )

        async with receiver:
            received_msgs = await receiver.receive_messages(max_message_count=20)

            for msg in received_msgs:
                await process_message(msg, processed_messages)

    # After processing messages, check if all additional functions executed successfully
    try:
        await perform_additional_functions()
        # If all functions succeed, complete the messages
        for msg in processed_messages:
            await msg.complete()
    except Exception as e:
        print(f"Error: {e}")
        # If any function fails, abandon all messages
        for msg in processed_messages:
            await msg.abandon()

In [ ]:
if __name__ == "__main__":
    asyncio.run(receive_messages())